In [128]:
import pandas as pd
import itertools
from bs4 import BeautifulSoup
import requests
import sys
import datetime

## Cross Region with Cities

Create SQL databases

In [2]:
import sqlite3

conn = sqlite3.connect('region_mapping.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

In [3]:
df_regions=pd.read_csv("region_cities/regions.csv",error_bad_lines=False,sep=",")
df_departments=pd.read_csv("region_cities/departments.csv",error_bad_lines=False,sep=",")
df_cities=pd.read_csv("region_cities/cities.csv",error_bad_lines=False,sep=",")

In [4]:
print("Region columns:", df_regions.columns)
print("Departments columns:", df_departments.columns)
print("Cities columns:", df_cities.columns)

Region columns: Index(['id', 'code', 'name', 'slug'], dtype='object')
Departments columns: Index(['id', 'region_code', 'code', 'name', 'slug'], dtype='object')
Cities columns: Index(['id', 'department_code', 'insee_code', 'zip_code', 'name', 'slug',
       'gps_lat', 'gps_lng'],
      dtype='object')


In [38]:
#DROP
c.execute('''DROP TABLE CITY''')
c.execute('''DROP TABLE DEPARTMENT''')
c.execute('''DROP TABLE REGION''')
# Create table - CITY
c.execute('''CREATE TABLE CITY
             ([id] INTEGER PRIMARY KEY,[department_code] int, [insee_code] int, [zip_code] int,[name] text,
             [slug] text, [gps_lat] text, [gps_lng] text)''')
          
# Create table - DEPARTMENT
c.execute('''CREATE TABLE DEPARTMENT
             ([id] INTEGER PRIMARY KEY,[region_code] int, [code] int, [name] text, [slug] text)''')
        
# Create table - REGION
c.execute('''CREATE TABLE REGION
             ([id] INTEGER PRIMARY KEY, [code] int, [name] text, [slug] text)''')

conn.commit()

In [39]:
df_regions.to_sql('REGION', conn, if_exists='append', index = False) 
df_cities.to_sql('CITY', conn, if_exists='append', index = False) 
df_departments.to_sql('DEPARTMENT', conn, if_exists='append', index = False) 

Select distinct regions

In [76]:
c.execute('''SELECT c.name,c.insee_code,r.name FROM CITY c JOIN DEPARTMENT d ON d.id=c.department_code JOIN REGION r ON d.region_code=r.code''')
results = c.fetchall()

In [81]:
df_selection=pd.DataFrame(results)
df_selection.columns=['name','insee','Région']

In [82]:
df_selection['Région'].unique()

array(['Auvergne-Rhône-Alpes', 'Hauts-de-France',
       "Provence-Alpes-Côte d'Azur", 'Grand Est', 'Occitanie',
       'Normandie', 'Nouvelle-Aquitaine', 'Centre-Val de Loire',
       'Bretagne', 'Bourgogne-Franche-Comté', 'Corse', 'Pays de la Loire',
       'Île-de-France'], dtype=object)

In [83]:
df_selection=df_selection[df_selection['Région']=='Île-de-France']
df_selection.head()

,name,insee,Région
29545,Allouville-Bellefosse,76001,Île-de-France
29546,Alvimare,76002,Île-de-France
29547,Ambrumesnil,76004,Île-de-France
29548,Amfreville-la-Mi-Voie,76005,Île-de-France
29549,Amfreville-les-Champs,76006,Île-de-France


Retrieve pop by cities

In [103]:
''''
def feed_pop(insee):
    print(insee.type)
    URL='https://geo.api.gouv.fr/communes/'+ str(insee)
    r = requests.get(url = URL)
    if r.status_code==200:
        data = r.json()
        return data['population']
    else:
        return 0
'''

In [105]:
df_selection['pop']=''
for index, row in df_selection.iterrows():
    URL='https://geo.api.gouv.fr/communes/'+ str(row['insee'])
    r = requests.get(url = URL)
    if r.status_code==200:
        data = r.json()
        df_selection['pop'][df_selection['name']==row['name']]=data['population']
    else:
        pass

/home/guillaume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [106]:
df_selection.head()

,name,insee,Région,pop
29545,Allouville-Bellefosse,76001,Île-de-France,1157
29546,Alvimare,76002,Île-de-France,620
29547,Ambrumesnil,76004,Île-de-France,480
29548,Amfreville-la-Mi-Voie,76005,Île-de-France,3222
29549,Amfreville-les-Champs,76006,Île-de-France,171


In [107]:
df_selection.to_csv('selection.csv',index=False)

## Cross cities with consommation

We have consommation by region, we assume that the production is equally dispatched into a same region since we could not retrieve more accurate data on how production is divided by cities/communes

In [125]:
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')

In [126]:
df_consommation=pd.read_csv("consommation-production/consommation-quotidienne-brute-regionale.csv",error_bad_lines=False,sep=";", parse_dates=['Date'], date_parser=dateparse)
df_consommation=df_consommation.sort_values(by=['Date', 'Heure','Région'])
df_consommation=df_consommation[df_consommation['Région']=='Ile-de-France']
#df_consommation=df_consommation[df_consommation['Date']>]

In [130]:
df_date=df_consommation[df_consommation['Date']>datetime.date(2017,11,22)]

/home/guillaume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [136]:
df_date.columns

Index(['Date - Heure', 'Date', 'Heure', 'Code INSEE région', 'Région',
       'Consommation brute gaz (MW PCS 0°C) - GRTgaz', 'Statut - GRTgaz',
       'Consommation brute gaz (MW PCS 0°C) - Teréga', 'Statut - Teréga',
       'Consommation brute gaz totale (MW PCS 0°C)',
       'Consommation brute électricité (MW) - RTE', 'Statut - RTE',
       'Consommation brute totale (MW)'],
      dtype='object')

In [143]:
df_date.sort_values(by=['Date', 'Heure','Région'],ascending=False)

,Date - Heure,Date,Heure,Région,Consommation brute gaz (MW PCS 0°C) - Teréga,Statut - Teréga,Consommation brute gaz totale (MW PCS 0°C),Consommation brute électricité (MW) - RTE,Statut - RTE,Consommation brute totale (MW)
415831,2017-12-31T23:30:00+01:00,2017-12-31,23:30,Ile-de-France,NaN,NaN,NaN,8406.0,Définitif,NaN
489069,2017-12-31T23:00:00+01:00,2017-12-31,23:00,Ile-de-France,NaN,NaN,11370.0,8676.0,Définitif,20046.0
633794,2017-12-31T21:30:00+01:00,2017-12-31,21:30,Ile-de-France,NaN,NaN,NaN,8052.0,Définitif,NaN
163905,2017-12-31T20:00:00+01:00,2017-12-31,20:00,Ile-de-France,NaN,NaN,13392.0,8779.0,Définitif,22171.0
597724,2017-12-31T19:30:00+01:00,2017-12-31,19:30,Ile-de-France,NaN,NaN,NaN,8888.0,Définitif,NaN
122702,2017-12-31T19:00:00+01:00,2017-12-31,19:00,Ile-de-France,NaN,NaN,14094.0,8898.0,Définitif,22992.0
172342,2017-12-31T18:30:00+01:00,2017-12-31,18:30,Ile-de-France,NaN,NaN,NaN,8825.0,Définitif,NaN
34806,2017-12-31T17:00:00+01:00,2017-12-31,17:00,Ile-de-France,NaN,NaN,13593.0,8127.0,Définitif,21720.0
172330,2017-12-31T16:30:00+01:00,2017-12-31,16:30,Ile-de-France,NaN,NaN,NaN,7884.0,Définitif,NaN
34805,2017-12-31T16:00:00+01:00,2017-12-31,16:00,Ile-de-France,NaN,NaN,13879.0,7903.0,Définitif,21782.0


In [146]:
df_final=df_date[['Date','Heure','Région','Consommation brute électricité (MW) - RTE']]
df_final.to_csv('final.csv',index=False)

In [109]:
df_consommation.dtypes

Date - Heure                                     object
Date                                             object
Heure                                            object
Code INSEE région                                 int64
Région                                           object
Consommation brute gaz (MW PCS 0°C) - GRTgaz    float64
Statut - GRTgaz                                  object
Consommation brute gaz (MW PCS 0°C) - Teréga    float64
Statut - Teréga                                  object
Consommation brute gaz totale (MW PCS 0°C)      float64
Consommation brute électricité (MW) - RTE       float64
Statut - RTE                                     object
Consommation brute totale (MW)                  float64
dtype: object

In [110]:
#df_selection=df_selection['Région'].replace('Île-de-France','Ile-de-France')
df_selection['Région']=df_selection['Région'].replace('Île-de-France','Ile-de-France')
df_selection.head()

,name,insee,Région,pop
29545,Allouville-Bellefosse,76001,Ile-de-France,1157
29546,Alvimare,76002,Ile-de-France,620
29547,Ambrumesnil,76004,Ile-de-France,480
29548,Amfreville-la-Mi-Voie,76005,Ile-de-France,3222
29549,Amfreville-les-Champs,76006,Ile-de-France,171


In [112]:
df_selection.dtypes

name      object
insee      int64
Région    object
pop       object
dtype: object

In [ ]:
df_2.set_index('specialites'), on='spe_2')

In [116]:
df_merge=df_selection.join(df_consommation.set_index('Région'), on='Région')
#df_merge['conso_ville']=df_merge['Consommation brute électricité (MW) - RTE']*df_merge('pop')/df.groupby(['Région'])['pop'].sum()

In [118]:
df_merge.sort_values(by=['name','Date', 'Heure'])

,name,insee,Région,pop,Date - Heure,Date,Heure,Code INSEE région,Consommation brute gaz (MW PCS 0°C) - GRTgaz,Statut - GRTgaz,Consommation brute gaz (MW PCS 0°C) - Teréga,Statut - Teréga,Consommation brute gaz totale (MW PCS 0°C),Consommation brute électricité (MW) - RTE,Statut - RTE,Consommation brute totale (MW)
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T00:00:00+01:00,01/01/2013,00:00,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T01:30:00+01:00,01/01/2013,01:30,11,NaN,NaN,NaN,NaN,NaN,8499.0,Définitif,NaN
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T03:00:00+01:00,01/01/2013,03:00,11,NaN,NaN,NaN,NaN,NaN,7848.0,Définitif,NaN
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T03:30:00+01:00,01/01/2013,03:30,11,NaN,NaN,NaN,NaN,NaN,7667.0,Définitif,NaN
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T04:30:00+01:00,01/01/2013,04:30,11,NaN,NaN,NaN,NaN,NaN,7378.0,Définitif,NaN
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T06:30:00+01:00,01/01/2013,06:30,11,NaN,NaN,NaN,NaN,NaN,7325.0,Définitif,NaN
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T07:00:00+01:00,01/01/2013,07:00,11,2869.0,Définitif,NaN,NaN,2869.0,7312.0,Définitif,10181.0
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T08:00:00+01:00,01/01/2013,08:00,11,2887.0,Définitif,NaN,NaN,2887.0,7402.0,Définitif,10289.0
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T08:30:00+01:00,01/01/2013,08:30,11,NaN,NaN,NaN,NaN,NaN,7479.0,Définitif,NaN
35046,Ableiges,95002,Ile-de-France,1149,2013-01-01T09:00:00+01:00,01/01/2013,09:00,11,2900.0,Définitif,NaN,NaN,2900.0,7417.0,Définitif,10317.0


In [119]:
df_merge.columns

Index(['name', 'insee', 'Région', 'pop', 'Date - Heure', 'Date', 'Heure',
       'Code INSEE région', 'Consommation brute gaz (MW PCS 0°C) - GRTgaz',
       'Statut - GRTgaz', 'Consommation brute gaz (MW PCS 0°C) - Teréga',
       'Statut - Teréga', 'Consommation brute gaz totale (MW PCS 0°C)',
       'Consommation brute électricité (MW) - RTE', 'Statut - RTE',
       'Consommation brute totale (MW)'],
      dtype='object')